In [8]:
# 计算message 和string中的token数量
from pyexpat.errors import messages
import tiktoken
# 2. Load an encoding 使用tiktoken.get_encoding() 按名称加载编码。第一次运行时，它将需要互联网连接进行下载。后续运行不需要互联网连接。
encoding = tiktoken.get_encoding("cl100k_base")
# 使用tiktoken.encoding_for_model()函数可以自动加载给定模型名称的正确编码。
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
# 3. Turn text into tokens with encoding.encode() The .encode() method converts a text string into a list of token integers.
encoding.encode("tiktoken is great!")
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """返回文本字符串中的Token数量"""
    num_tokens=0
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    num_tokens = len(encoding.encode(string))
    return num_tokens
def calculate_messages(messages:list) -> int:
    """返回总的message长度"""
    counter=0
    for mess in messages:
        counter+=num_tokens_from_string(mess.content,"cl100k_base")
    return counter
INCONTEXT_WINDOWS_GPT_3_5=16000
INCONTEXT_WINDOWS_MOONSHOT=30000

In [22]:
# from dotenv import load_dotenv

#  现在所有的ChatGPT 系列都是16,385
# # 16,385
# long_chat =ChatOpenAI(temperature=0,model="gpt-3.5-turbo-16k")

In [23]:
# moonshot_chat.openai_api_base

In [24]:
# moonshot_chat

``` python
"""
messages = [
    SystemMessage(
        content="我希望你扮演一个中文语法和心理咨询专家，你需要根据语境纠正下列文本中的OCR的语法错误和中文符号使用错误。我希望你以json格式输出纠正错误OCR识别后的结果。json元素的键值包括：'id','participant','text','reason',其中'id','participant' 保留原始数据，不得修改，如果text中有语法或符号错误，输出修改后结果。'reason'记录修改原因，如果没有修改，reason值为空。请不要大幅度修改句子结构，主要修改错词和错误的中文标点符号。下面为部分例子，请确保输出格式为标准的json格式："
    ),
    HumanMessage(
        content=r"""请你帮我纠正下面这段心理咨询对话OCR后的结果，以json数组形式输入，以json数组的形式输出:"""+input_str
    ),
    AIMessage(content=output_str),
    HumanMessage(
        content=r"""请你帮我纠正下面这段心理咨询对话OCR后的结果，以json数组形式输入，以json数组的形式输出:"""+input_str2
    ),
    AIMessage(
        content=output_str2
    ),
    HumanMessage(content=r"""请你帮我纠正下面这段心理咨询对话OCR后的结果，以json数组形式输入，以json数组的形式输出:"""+input_str3),
    AIMessage(
        content=output_str3
    ),
    HumanMessage(content="请你帮我纠正下面这段心理咨询对话OCR后的结果，以json数组形式输入，以json数组的形式输出:"+json.dumps(cn_dialog_data[315]['dialog_list'],ensure_ascii=False))
]
"""
```

In [9]:
import os
import json

fewshot_list=[]
fewshot_dir_path="/home/ckqsudo/code2024/0dataset/E-bed/English/z3_gpt_mapping.ipynb/提示例子/fewshot"
fewshot_file_list=os.listdir(fewshot_dir_path)
res_dict={}
for file_name in fewshot_file_list:
    with open(fewshot_dir_path+"/"+file_name,"r",encoding="utf-8") as i_f:
        prompt_data=json.load(i_f)
    prompt_id=file_name.split("_")[0]
    prompt_key=file_name.split("_")[-1].split(".")[0]
    if prompt_id not in res_dict:
        res_dict[prompt_id]={"original":None,"processed":None}
    
    if prompt_key=="original":
        res_dict[prompt_id]["original"]=prompt_data
    elif prompt_key=="process":
        res_dict[prompt_id]["processed"]=prompt_data
    else:
        print("文件命名出错")

In [10]:
# res=sorted(res,dict(sorted(my_dict.items(), key=lambda item: item[1])))
res_dict=dict(sorted(res_dict.items(),key=lambda item:int(item[0])))
res_list=list(res_dict.values())

In [11]:
len(res_dict)

3

In [28]:
# res_list[1]

In [12]:
import os
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage,AIMessage
from langchain_openai import ChatOpenAI

import os
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage,AIMessage
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(temperature=0,model="gpt-3.5-turbo",openai_api_key='XXX',openai_api_base='https://gptnb.keqichen.top/v1')
# chat.openai_api_key=
# chat.openai_api_base=
# moonshot_chat.openai_api_key=
# moonshot_chat.openai_api_base="https://api.moonshot.cn/v1"
moonshot_chat=ChatOpenAI(temperature=0,model="moonshot-v1-32k",openai_api_key='xxx',openai_api_base='https://api.moonshot.cn/v1')
# moonshot_chat.temperature


In [13]:

from z_prompt_for_gpt import prefix_example_post_rewrite,prefix_instruction,system_prompt
def get_message(cur_input:str,original_processed_pair:list):
    messages = [
    SystemMessage(
        content=system_prompt
    )]
    for item in original_processed_pair:
        messages.append(HumanMessage(
        content="这是一个例子："+item["original_shot"]
        ))
        messages.append(AIMessage(
        content=prefix_example_post_rewrite+item["processed_shot"]))
    
    messages.append(HumanMessage(
        content=prefix_instruction+cur_input
    ))
    token_count=calculate_messages(messages)
    print(len(messages),token_count)
    return messages,token_count

In [33]:
test=[['', '104_cache.json'],
 ["JSONDecodeError('Expecting value: line 1 column 1 (char 0)')",
  '227_cache.json'],
 ["JSONDecodeError('Expecting property name enclosed in double quotes: line 19 column 1 (char 1124)')",
  '236_cache.json'],
 ["JSONDecodeError('Expecting property name enclosed in double quotes: line 37 column 1 (char 3052)')",
  '23_cache.json'],
 ["JSONDecodeError(Expecting ',' delimiter: line 14 column 26 (char 811)')",
  '252_cache.json'],
 ["JSONDecodeError('Expecting property name enclosed in double quotes: line 73 column 1 (char 4405)')",
  '257_cache.json'],
 ["Expecting ',' delimiter: line 79 column 5 (char 5194)", '265_cache.json'],
 ['Expecting value: line 52 column 1 (char 4757)', '287_cache.json'],
 ['Expecting property name enclosed in double quotes: line 31 column 1 (char 2012)',
  '308_cache.json'],
 ['Expecting property name enclosed in double quotes: line 25 column 1 (char 1935)',
  '310_cache.json'],
 ['Extra data: line 54 column 1 (char 4582)', '315_cache.json'],
 ['Expecting value: line 1 column 1 (char 0)', '355_cache.json'],
 ['Expecting property name enclosed in double quotes: line 30 column 13 (char 1590)',
  '380_cache.json'],
 ['Expecting property name enclosed in double quotes: line 28 column 1 (char 2094)',
  '422_cache.json'],
 ['Expecting value: line 1 column 1 (char 0)', '475_cache.json'],
 ["Expecting ',' delimiter: line 139 column 63 (char 5901)", '485_cache.json'],
 ['Expecting property name enclosed in double quotes: line 586 column 16 (char 19296)',
  '519_cache.json'],
 ['Expecting property name enclosed in double quotes: line 58 column 1 (char 2414)',
  '526_cache.json']]
for a,b in test:
    print(b)

104_cache.json
227_cache.json
236_cache.json
23_cache.json
252_cache.json
257_cache.json
265_cache.json
287_cache.json
308_cache.json
310_cache.json
315_cache.json
355_cache.json
380_cache.json
422_cache.json
475_cache.json
485_cache.json
519_cache.json
526_cache.json


In [44]:
import json
import random
import time
def data_handler(input_dict:dict,input_file_name:str,output_cache_path:str,error_list:list,error_list_file_path:str,handle_error=False):
    prompt_idx=random.randint(0,len(res_list)-1)
    cur_input=json.dumps(input_dict,ensure_ascii=False,indent=4)
    original_processed_pair=[
        {"original_shot": json.dumps(res_list[prompt_idx]["original"],ensure_ascii=False,indent=4),
        "processed_shot": json.dumps(res_list[prompt_idx]["processed"],ensure_ascii=False,indent=4)}]
    messages,token_count=get_message(cur_input,original_processed_pair)

    if token_count<3000:
        next_idx=0
        while next_idx==prompt_idx:
            next_idx=random.randint(0,len(res_list)-1)
        original_processed_pair.append({"original_shot": json.dumps(res_list[next_idx]["original"],ensure_ascii=False,indent=4),
        "processed_shot": json.dumps(res_list[next_idx]["processed"],ensure_ascii=False,indent=4)})
        messages,token_count=get_message(cur_input,original_processed_pair)
    try:
        if token_count>9000:
            error_list.append(["",input_file_name])
            return
        elif token_count>INCONTEXT_WINDOWS_GPT_3_5-8000:
            cur_chat=moonshot_chat
            print(f"cur_chat=moonshot_chat{file_name}_{handle_error}")
        else:
            cur_chat=chat
            print(f"cur_chat=chat{file_name}_{handle_error}")
        if handle_error==True:
            cur_chat=moonshot_chat
            cur_chat.temperature=0.2
            print(f"cur_chat=moonshot_chat{file_name}_{handle_error}")
        response=cur_chat(messages)
        print(f"{input_file_name}_chat_finish")
        response_json=json.loads(response.content)
        if "background" not in response_json:
            response_json["background"]="Unknown"
        ouput_cache_file_path=output_cache_path+"/"+input_file_name
        if os.path.exists(ouput_cache_file_path)==False:
            with open(ouput_cache_file_path,"w",encoding="utf-8") as o_f:
                json.dump(response_json,o_f,ensure_ascii=False)
        else:
            print("文件已存在")
    except Exception as e:
        error_list.append([str(e),input_file_name])
        with open(error_list_file_path,"w",encoding="utf-8") as log_o_file:
            json.dump(error_list,log_o_file,ensure_ascii=False)
        print(error_list,"出现错误，需要单独处理",response.content)
        time.sleep(10)
        return
    time.sleep(2)
# test
# with open("/home/ckqsudo/code2024/0dataset/E-bed/English/z3_gpt_mapping.ipynb/缓存文件夹/3_cache.json","r",encoding="utf-8") as i_f:
#     en_data=json.load(i_f)
#     data_handler(input_dict=en_data,input_file_name="3_cache.json",output_cache_path="/home/ckqsudo/code2024/0dataset/E-bed/English/z3_gpt_mapping.ipynb/缓存输出文件夹",error_file_list=[])

In [41]:
error_list
[['', '104_cache.json'],
 ["JSONDecodeError('Expecting value: line 1 column 1 (char 0)')",
  '227_cache.json'],
 ["JSONDecodeError('Expecting property name enclosed in double quotes: line 19 column 1 (char 1124)')",
  '236_cache.json'],
 ["JSONDecodeError('Expecting property name enclosed in double quotes: line 37 column 1 (char 3052)')",
  '23_cache.json'],
 ["JSONDecodeError(Expecting ',' delimiter: line 14 column 26 (char 811)')",
  '252_cache.json'],
 ["JSONDecodeError('Expecting property name enclosed in double quotes: line 73 column 1 (char 4405)')",
  '257_cache.json'],
 ["Expecting ',' delimiter: line 79 column 5 (char 5194)", '265_cache.json'],
 ['Expecting value: line 52 column 1 (char 4757)', '287_cache.json'],
 ['Expecting property name enclosed in double quotes: line 31 column 1 (char 2012)',
  '308_cache.json'],
 ['Expecting property name enclosed in double quotes: line 25 column 1 (char 1935)',
  '310_cache.json'],
 ['Extra data: line 54 column 1 (char 4582)', '315_cache.json'],
 ['Expecting value: line 1 column 1 (char 0)', '355_cache.json'],
 ['Expecting property name enclosed in double quotes: line 30 column 13 (char 1590)',
  '380_cache.json'],
 ['Expecting property name enclosed in double quotes: line 28 column 1 (char 2094)',
  '422_cache.json'],
 ['Expecting value: line 1 column 1 (char 0)', '475_cache.json'],
 ["Expecting ',' delimiter: line 139 column 63 (char 5901)", '485_cache.json'],
 ['Expecting property name enclosed in double quotes: line 586 column 16 (char 19296)',
  '519_cache.json'],
 ['Expecting property name enclosed in double quotes: line 58 column 1 (char 2414)',
  '526_cache.json']]

[['', '104_cache.json'],
 ["JSONDecodeError('Expecting value: line 1 column 1 (char 0)')",
  '227_cache.json'],
 ["JSONDecodeError('Expecting property name enclosed in double quotes: line 19 column 1 (char 1124)')",
  '236_cache.json'],
 ["JSONDecodeError('Expecting property name enclosed in double quotes: line 37 column 1 (char 3052)')",
  '23_cache.json'],
 ["JSONDecodeError(Expecting ',' delimiter: line 14 column 26 (char 811)')",
  '252_cache.json'],
 ["JSONDecodeError('Expecting property name enclosed in double quotes: line 73 column 1 (char 4405)')",
  '257_cache.json'],
 ["Expecting ',' delimiter: line 79 column 5 (char 5194)", '265_cache.json'],
 ['Expecting value: line 52 column 1 (char 4757)', '287_cache.json'],
 ['Expecting property name enclosed in double quotes: line 31 column 1 (char 2012)',
  '308_cache.json'],
 ['Expecting property name enclosed in double quotes: line 25 column 1 (char 1935)',
  '310_cache.json'],
 ['Extra data: line 54 column 1 (char 4582)', '315_cache

In [5]:
import os
import json
cache_floder="/home/ckqsudo/code2024/0dataset/E-bed/English/z3_gpt_mapping.ipynb/缓存文件夹"
output_cache_path="/home/ckqsudo/code2024/0dataset/E-bed/English/z3_gpt_mapping.ipynb/缓存输出文件夹"
log_json_path="/home/ckqsudo/code2024/0dataset/E-bed/English/z3_gpt_mapping.ipynb/error_list.json"
file_name_list=os.listdir(cache_floder)
output_cache_file_list=os.listdir(output_cache_path)
# 
error_list_file_path="/home/ckqsudo/code2024/0dataset/E-bed/English/z3_gpt_mapping.ipynb/error_list.json"

In [39]:
# output_cache_file_list

In [46]:

file_name_list.sort()
with open(log_json_path,"r",encoding="utf-8") as i_f:
    error_list=json.load(i_f)
# error_list=set(list)
def main(handle_error=False):
    for file_name in file_name_list:
        if file_name  in output_cache_file_list:
            if handle_error and file_name in [item[-1] for item in error_list]:
                print(f"处理错误文件{file_name}")
            else:
                print(f"已生成,跳过{file_name}")
                continue
        file_name_path=cache_floder+"/"+file_name
        with open(file_name_path,"r",encoding="utf-8") as i_f:
            en_data=json.load(i_f)
            data_handler(input_dict=en_data,input_file_name=file_name,output_cache_path=output_cache_path,error_list=error_list,error_list_file_path=error_list_file_path,handle_error=handle_error)
main()

已生成,跳过0_cache.json
已生成,跳过100_cache.json
已生成,跳过101_cache.json
已生成,跳过102_cache.json
已生成,跳过103_cache.json
4 10687
已生成,跳过105_cache.json
已生成,跳过106_cache.json
已生成,跳过107_cache.json
已生成,跳过108_cache.json
已生成,跳过109_cache.json
已生成,跳过10_cache.json
已生成,跳过110_cache.json
已生成,跳过111_cache.json
已生成,跳过112_cache.json
已生成,跳过113_cache.json
已生成,跳过114_cache.json
已生成,跳过115_cache.json
已生成,跳过116_cache.json
已生成,跳过117_cache.json
已生成,跳过118_cache.json
已生成,跳过119_cache.json
已生成,跳过11_cache.json
已生成,跳过120_cache.json
已生成,跳过121_cache.json
已生成,跳过122_cache.json
已生成,跳过123_cache.json
已生成,跳过124_cache.json
已生成,跳过125_cache.json
已生成,跳过126_cache.json
已生成,跳过127_cache.json
已生成,跳过128_cache.json
已生成,跳过129_cache.json
已生成,跳过12_cache.json
已生成,跳过130_cache.json
已生成,跳过131_cache.json
已生成,跳过132_cache.json
已生成,跳过133_cache.json
已生成,跳过134_cache.json
已生成,跳过135_cache.json
已生成,跳过136_cache.json
已生成,跳过137_cache.json
已生成,跳过138_cache.json
已生成,跳过139_cache.json
已生成,跳过13_cache.json
已生成,跳过140_cache.json
已生成,跳过141_cache.json
已生成,跳过142_cache.json
已生成,跳过143_c

/home/ckqsudo/sys_tool/miniconda/tmp/yes/envs/trl/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


227_cache.json_chat_finish
[['', '104_cache.json'], ["JSONDecodeError('Expecting value: line 1 column 1 (char 0)')", '227_cache.json'], ["JSONDecodeError('Expecting property name enclosed in double quotes: line 19 column 1 (char 1124)')", '236_cache.json'], ["JSONDecodeError('Expecting property name enclosed in double quotes: line 37 column 1 (char 3052)')", '23_cache.json'], ["JSONDecodeError(Expecting ',' delimiter: line 14 column 26 (char 811)')", '252_cache.json'], ["JSONDecodeError('Expecting property name enclosed in double quotes: line 73 column 1 (char 4405)')", '257_cache.json'], ["Expecting ',' delimiter: line 79 column 5 (char 5194)", '265_cache.json'], ['Expecting value: line 52 column 1 (char 4757)', '287_cache.json'], ['Expecting property name enclosed in double quotes: line 31 column 1 (char 2012)', '308_cache.json'], ['Expecting property name enclosed in double quotes: line 25 column 1 (char 1935)', '310_cache.json'], ['Extra data: line 54 column 1 (char 4582)', '315_ca

In [35]:
# 原地修改
file_list_in_cache_input=os.listdir(cache_floder)
file_list_in_cache_output=os.listdir(output_cache_path)
# for file in sorted(file_list_in_cache_output):
#     with open(output_cache_path+"/"+file,"r",encoding="utf-8") as i_f:
#         output_json=json.load(i_f)
#     if "background" not in output_json:
#         output_json["background"]="Unknown"
#         print(file)
#         with open(output_cache_path+"/"+file,"w",encoding="utf-8") as o_f:
#             json.dump(output_json,o_f,ensure_ascii=False)

In [36]:
set(file_list_in_cache_input)-set()

{'104_cache.json', '519_cache.json'}

In [15]:

inequal_list=[]
for file in sorted(file_list_in_cache_output):
    # print(file)
    with open(cache_floder+"/"+file,"r",encoding="utf-8") as i_0f:
        input_json=json.load(i_0f)
    with open(output_cache_path+"/"+file,"r",encoding="utf-8") as o_f:
        output_json=json.load(o_f)
        # 
    if len(input_json)==len(output_json)-1 and len(input_json["topic_dialog"])==len(output_json["topic_dialog"]) and input_json["topic_dialog"]["dialog"][-1]["id"]==output_json["topic_dialog"]["dialog"][-1]["id"]:
        pass
    else:
        # print(file," ",type(input_json["topic_dialog"]["dialog"]),type(input_json["topic_dialog"]["dialog"]))
        # print("in",len(input_json),len(input_json["topic_dialog"]))
        # print("out",len(output_json),len(output_json["topic_dialog"]))
        # print(input_json)
        # print(output_json.keys())
        # print(set(list(output_json.keys())+["background"])-set(input_json.keys()))
        # print(len(input_json["topic_dialog"]["dialog"]))
        # print(input_json["topic_dialog"]["dialog"][-1]["id"])
        # print(len(output_json["topic_dialog"]["dialog"]))
        # print(output_json["topic_dialog"]["dialog"][-1]["id"])
        # break
        inequal_list.append(file)
    # print(len(inequal_list))

In [16]:
re_process_list=inequal_list+[item[-1] for item in error_list]
list(set(re_process_list))

NameError: name 'error_list' is not defined

In [34]:
# re_process_list=['310_cache.json',
#  '475_cache.json',
#  '23_cache.json',
#  '183_cache.json',
#  '114_cache.json',
#  '104_cache.json',
#  '227_cache.json',
#  '194_cache.json',
#  '94_cache.json',
#  '267_cache.json',
#  '3_cache.json',
#  '315_cache.json',
#  '380_cache.json',
#  '519_cache.json',
#  '58_cache.json',
#  '287_cache.json',
#  '485_cache.json',
#  '245_cache.json',
#  '463_cache.json',
#  '193_cache.json',
#  '295_cache.json',
#  '16_cache.json',
#  '562_cache.json',
#  '526_cache.json',
#  '554_cache.json',
#  '191_cache.json',
#  '252_cache.json',
#  '8_cache.json',
#  '263_cache.json',
#  '487_cache.json',
#  '529_cache.json',
#  '507_cache.json',
#  '12_cache.json',
#  '327_cache.json',
#  '2_cache.json',
#  '209_cache.json',
#  '537_cache.json',
#  '257_cache.json',
#  '76_cache.json',
#  '103_cache.json',
#  '424_cache.json',
#  '101_cache.json',
#  '355_cache.json',
#  '102_cache.json',
#  '510_cache.json',
#  '488_cache.json',
#  '77_cache.json',
#  '422_cache.json',
#  '176_cache.json',
#  '366_cache.json',
#  '153_cache.json',
#  '236_cache.json',
#  '265_cache.json',
#  '308_cache.json']

In [21]:
# "aaf"[-4:]

'aaf'

In [31]:
import json
import random
import time
def re_process_main(input_dict:dict,input_file_name:str,output_cache_path:str,error_list:list,error_list_file_path:str,is_overwrite:False):
    have_processed=[]
    prompt_idx=0
    original_processed_pair=[
        {"original_shot": json.dumps(res_list[prompt_idx]["original"],ensure_ascii=False,indent=4),
        "processed_shot": json.dumps(res_list[prompt_idx]["processed"],ensure_ascii=False,indent=4)
        }]
    cur_input=json.dumps(input_dict,ensure_ascii=False,indent=4)
    messages,token_count=get_message(cur_input,original_processed_pair)
    
    try:
        cur_chat=chat
        cur_chat.temperature=0.1
        response=cur_chat(messages)
        counter=0
        whole_str=response.content
        while str(response.content).endswith("}") and counter<3:
            time.sleep(10)
            print(f"{str(response.content)[-4]}")
            messages.append(AIMessage(content=response.content))
            messages.append(HumanMessage(content="这是你原先输出的结果,但是是不完整的,请将其补全："))
            cur_chat_content=cur_chat(messages)
            whole_str+=cur_chat_content.content   
            counter+=1
        if counter>=3:
            raise ValueError("max_try")
        print(whole_str)
        response_json=json.loads(whole_str)
        if "background" not in response_json:
            response_json["background"]="Unknown"
        ouput_cache_file_path=output_cache_path+"/"+input_file_name
        if os.path.exists(ouput_cache_file_path)==False or is_overwrite:
            with open(ouput_cache_file_path,"w",encoding="utf-8") as o_f:
                json.dump(response_json,o_f,ensure_ascii=False)
            print(f"{input_file_name} save_finish")
            have_processed.append(input_file_name)
        else:
            print("文件已存在")
    except Exception as e:
        with open("/home/ckqsudo/code2024/0dataset/E-bed/English/z3_gpt_mapping.ipynb/txt缓存文件夹"+"/"+input_file_name+".txt","w",encoding="utf-8") as txt_f:
                txt_f.write(whole_str)
        print(f"{input_file_name} txt_save_finish")
        error_list.append([str(e),input_file_name])
        with open(error_list_file_path,"w",encoding="utf-8") as log_o_file:
            json.dump(error_list,log_o_file,ensure_ascii=False)
        print(error_list,f"已经处理列表{have_processed}")
        time.sleep(10)
        return
    time.sleep(2)

In [32]:
new_error_list=[]
def re_process(handle_error=True):
    # error_list_file_path
    for file_name in sorted(re_process_list):
        file_name_path=cache_floder+"/"+file_name
        with open(file_name_path,"r",encoding="utf-8") as i_f:
            en_data=json.load(i_f)
            re_process_main(input_dict=en_data,input_file_name=file_name,output_cache_path=output_cache_path,error_list=new_error_list,error_list_file_path=error_list_file_path,is_overwrite=True)
re_process(True)
# re_process_list

4 5445
{
    "case\_theme": "Solution-Focused Brief Therapy",
    "background": "A school social worker is trying to help a student who has experienced neglect and abuse by building on the student's strengths and abilities.",
    "pre\_reasoning": "The school social worker is not engaging in treatment but building on the student's strengths and abilities to thrive and do as best she can. Using scaling and having the student focus on what she thinks the teacher would rate her is intentional. The effort is to strengthen the student's relationship with an important person who cares about her and believes in her ability. This is a necessary experience for a child who has been neglected and most likely feels abandoned by others. The very fact of being focused on a potentially positive effort and helping the student to build a stronger and supportive relationship with an adult is a significant factor in building the sense of trust and safety that is reparative of the unsafe experiences in he

UnboundLocalError: local variable 'whole_str' referenced before assignment

In [79]:
len([{"A":"b","c":"d"}])

1

In [74]:
len(inequal_list)

557

In [61]:
file_list_in_cache_input=os.listdir(cache_floder)
file_list_in_cache_output=os.listdir(output_cache_path)
unhandle_list=list(set(file_list_in_cache_output)-set(file_list_in_cache_input))

In [63]:
unhandle_list,list(set(file_list_in_cache_input)-set(file_list_in_cache_output))

([],
 ['315_cache.json',
  '519_cache.json',
  '252_cache.json',
  '287_cache.json',
  '104_cache.json',
  '227_cache.json'])

In [22]:
# error_file_list_output1=[["", "104_cache.json"], ["JSONDecodeError('Expecting value: line 1 column 1 (char 0)')", "227_cache.json"], ["JSONDecodeError('Expecting property name enclosed in double quotes: line 19 column 1 (char 1124)')", "236_cache.json"], ["JSONDecodeError('Expecting property name enclosed in double quotes: line 37 column 1 (char 3052)')", "23_cache.json"], ["JSONDecodeError(Expecting ',' delimiter: line 14 column 26 (char 811)')", "252_cache.json"], ["JSONDecodeError('Expecting property name enclosed in double quotes: line 73 column 1 (char 4405)')", "257_cache.json"], ["Expecting ',' delimiter: line 79 column 5 (char 5194)", "265_cache.json"], ["Expecting value: line 52 column 1 (char 4757)", "287_cache.json"], ["Expecting property name enclosed in double quotes: line 31 column 1 (char 2012)", "308_cache.json"], ["Expecting property name enclosed in double quotes: line 25 column 1 (char 1935)", "310_cache.json"], ["Extra data: line 54 column 1 (char 4582)", "315_cache.json"], ["Expecting value: line 1 column 1 (char 0)", "355_cache.json"], ["Expecting property name enclosed in double quotes: line 30 column 13 (char 1590)", "380_cache.json"], ["Expecting property name enclosed in double quotes: line 28 column 1 (char 2094)", "422_cache.json"], ["Expecting value: line 1 column 1 (char 0)", "475_cache.json"], ["Expecting ',' delimiter: line 139 column 63 (char 5901)", "485_cache.json"], ["Expecting property name enclosed in double quotes: line 586 column 16 (char 19296)", "519_cache.json"], ["Expecting property name enclosed in double quotes: line 58 column 1 (char 2414)", "526_cache.json"]]


In [16]:
test=[[0,1],[2,3]]
for a,b in test:
    print(a,b)

0 1
2 3
